In [1]:
# Imports
import json, os
import pandas as pd
from yelpapi import YelpAPI

In [2]:
# Load Credentials
with open('/Users/julia/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [17]:
# Set our API call parameters and filename before the first call
LOCATION = 'San Diego, CA'
TERM = 'sushi'

In [24]:
# Specifying JSON_FILE filename 
JSON_FILE = f"Data/results_in_progress_sushi.json"
JSON_FILE

'Data/results_in_progress_sushi.json'

In [6]:
# Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
# If it does not exist: 
if file_exists == False:
    
    # CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    # If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    # INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    # save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_pasta.json already exists.


In [19]:
# Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'{n_results} previous results found.')

956 previous results found.


In [20]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [21]:
# How many results total?
total_results = results['total']
total_results

1300

In [22]:
# How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [23]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

18

In [ ]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

In [ ]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)



In [ ]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

In [ ]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()

In [ ]:
final_df.drop_duplicates(inplace=True)

In [ ]:
# save the final results to a compressed csv
final_df.to_csv('Data/results_in_progress_sushi.csv.gz', compression='gzip',index=False)